# Basics of RSOME

RSOME, an open-source algebraic library, specializes in modeling generic optimization problems within uncertain environments. Offering a highly readable and mathematically intuitive modeling environment, RSOME is rooted in the state-of-the-art robust stochastic optimization framework.

This guide serves as an introduction to the principal components, fundamental data structures, and syntax rules inherent to the RSOME package. For installation details, please refer to the [](install.ipynb) page for comprehensive information.

(rsome_basics:modeling_env)= 
## Modeling Environments
The RSOME package encompasses two key modules designed for formulating optimization problems within uncertain contexts:

- The `ro` module is a specialized modeling framework tailored for robust optimization problems. It equips users with modeling tools explicitly crafted for creating uncertainty sets and defining affine decision rules in multi-stage decision-making scenarios.

- The `dro` module builds upon the distributionally robust optimization framework introduced by {cite:ps}`Chen_Sim_Xiong_2020robust`. This module offers modeling tools for constructing event-wise ambiguity sets and specifying event-wise adaptation policies.

(rsome_basics:intro_ro_env)= 
## Introduction to the `rsome.ro` Environment

### Models
In RSOME, all optimization models are specified based on a `Model` type object. Such an object is created by the constructor `Model()` imported from the `rsome.ro` modeling environment.

In [1]:
from rsome import ro            # import the ro modeling tool

model = ro.Model('My model')    # create a Model object named 'My model'

The code above defines a new `Model` object `model`, with the name specified to be the string `'My model'`. You could also leave the name unspecified and the default name is `None`.

### Decision Variables and Linear Constraints

To define decision variables as arrays for the previously established model, you can utilize the statement `model.dvar(shape, vtype)`. In this context, the `shape` parameter designates the array's dimensions, while `vtype` specifies whether the variables are continuous ('C'), binary ('B'), or general integers ('I'). If left unspecified, the variables default to continuous. For instance:

- `x` as a one-dimensional array with three integer decision variables;
- `y` as a $3\times 5$ array of 15 binary decision variables;
- `z` as a $2\times 3 \times 4 \times 5$ array of 120 continuous decision variables;

can be defined by the following code segment.

In [2]:
x = model.dvar(3, vtype='I')    # 3 integer variables as a 1D array
y = model.dvar((3, 5), 'B')     # 3x5 binary variables as a 2D array
z = model.dvar((2, 3, 4, 5))    # 2x3x4x5 continuous variables as a 4D array

In [3]:
import numpy as np

I, J = 7, 13

A = np.random.rand(J, I)
b = np.random.rand(I)
c = np.random.rand(J)

RSOME variables, defined as arrays, seamlessly integrate with standard NumPy array operations for expressing linear constraints. This includes capabilities for element-wise computation, matrix calculations, broadcasting, indexing, slicing, and more. This compatibility empowers users to define constraint blocks efficiently using the array system. For instance, the following constraint system:

```{math}
\begin{align}
&\sum\limits_{i\in[I]}b_ix_i = 1 && \\
&\sum\limits_{i\in[I]}A_{ji}x_i \leq c_j && j\in[J] \\
&\sum\limits_{j\in[J]}\sum\limits_{i\in I}y_{ji} \geq 1 &&\\
&\sum\limits_{i\in[I]}y_{ji} \geq 0 && j\in [J] \\
&A_{ji}x_i \geq 1 &&\forall j\in[J], i\in[I] \\
&A_{ji}y_{ji} + x_i \geq 0 && \forall j\in [J], i\in[I]
\end{align}
```

with decision variable $\pmb{x}\in\mathbb{R}^I$ and $\pmb{y}\in\mathbb{R}^{J\times I}$, along with parameters $\pmb{A}\in\mathbb{R}^{J\times I}$, $\pmb{b}\in\mathbb{R}^I$, and $\pmb{c}\in\mathbb{R}^J$, can be conveniently expressed using the code segment provided below.

In [4]:
x = model.dvar(I)               # define x as a 1D array of I variables
y = model.dvar((J, I))          # define y as a 2D array of JxI variables

b @ x == 1                      
A @ x <= c
y.sum() >= 1
y.sum(axis=1) >= 0
A * x >= 1
A*y + x >= 0

91 linear constraints

RSOME arrays can also be used in specifying the objective function of the optimization model. Note that the objective function must be one affine expression. In other words, the `size` attribute of the expression must be one, otherwise an error message would be generated.

```python
model.min(b @ x)        # minimize the objective function b @ x
model.max(b @ x)        # maximize the objective function b @ x
```

Model constraints can be specified by the method st(), which means "subject to". This method allows users to define their constraints in different ways.

In [5]:
model.st(A @ x <= c)                    # define one constraint

model.st(y.sum() >= 1,
         y.sum(axis=1) >= 0,
         A*y + x >= 0)                  # define multiple constraints

model.st(x[i] <= i for i in range(3))   # define constraints by a loop

<generator object <genexpr> at 0x136ee5700>

### Convex Expressions and Convex Constraints

The RSOME package also support a range of functions for specifying convex expressions and constraints. The definition and syntax of these functions closely parallel the universal functions found in the NumPy package, as outlined in the tables below.

|Function| Description|
|:-------|:--------------|
|`abs(x)`|The element-wise absolute values of `x`. |
|`entropy(x)`|The entropic expression `-sum(x * log(x))`. |
|`exp(x)`|The element-wise natural exponential of `x`. |
|`expcone(x, y, z)`| The exponential cone constraint `z * exp(x/z) <= y`. |
|`kldiv(p, q, r)`| The KL divergence constraint `sum(p*log(p/q)) <= r`. |
|`log(x)`|The element-wise natural logarithm of `x`.|
|`norm(x, degree)`| The $L^p$-norm of the `x`, where the value of $p$ is specified by `degree`.|
|`pexp(x, y)`|The element-wise perspective natural exponential `y * exp(x/y)`. |
|`plog(x, y)`|The element-wise perspective natural logarithm `y * log(x/y)`. |
|`quad(x, Q)`|The quadratic term `x @ Q @ x`. |
|`rsocone(x, y, z)`| The rotated conic constraint `sumsqr(x) <= y*z`. |
|`square(x)`|The element-wise squared values of `x`. |
|`sumsqr(x)`|The sum of squares of `x`. |


### Matrix Operations
When working with matrices (2-D arrays), RSOME facilitates fundamental operations such as calculating the trace, extracting diagonal elements, accessing upper/lower triangular elements, and concatenating arrays. Functions executing such operations are summarized in the following table. 

|Function&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Description|
|:-------|:--------------|
|`concat(arrays, axis)`| Concatenated arrays along the axes specified by `axis`.|
|`cstack(c1, ..., cn)`|An array formed by stacking the iven arrays along axis 1. |
|`diag(x, k, fill)`| The diagonal elements of a two-dimensional array `x`. |
|`rstack(r1, ..., rn)`|An array formed by stacking the given arrays along axis 0. |
|`trace(x)`| The trace of a 2-D array `x`. |
|`tril(x, k)`| The lower triangular elements of a two-dimensional array `x`. The integer `k` (`k=0` by default) specifies the shifts of the taken triangular elements.|
|`triu(x, k)`| The upper triangular elements of a two-dimensional array `x`. The integer `k` (`k=0` by default) specifies the shifts of the taken triangular elements.|

RSOME also supports defining semidefiniteness constraints or linear matrix inequalities that enforce an array `X` to be positive semidefinite (`X >> 0`) or negative semidefinite (`X << 0`). For instance, the constraint

```{math}
\left(
\begin{array}{cc}
A & Z \\
Z^{\top} & \text{diag}(Z)
\end{array}
\right) \succeq 0
```

with $A\in\mathbb{R}^{n\times n}$ and $Z$ being a lower triangular matrix, can be written as the following Python code,

In [6]:
n = 5

In [7]:
import rsome as rso
from rsome import ro

model = ro.Model()

A = model.dvar((n, n))
Z = rso.tril(model.dvar((n, n)))

model.st(rso.rstack([A, Z], 
                    [Z.T, rso.diag(Z, fill=True)]) >> 0)

10x10 linear matrix inequliaty constraint

or equivalently,

In [8]:
import rsome as rso
from rsome import ro

model = ro.Model()

A = model.dvar((n, n))
Z = rso.tril(model.dvar((n, n)))

model.st(rso.cstack([A, Z.T], 
                    [Z, rso.diag(Z, fill=True)]) >> 0)

10x10 linear matrix inequliaty constraint

In this example, the `T` attribute of an RSOME array is consistent with the transpose operations of NumPy arrays, functions `rstack()` and `cstack()` are used to create new arrays by stacking rows or columns together, respectively. 

(rsome_basics:sform_sol)= 
## Standard Forms and Solutions 

Every RSOME model undergoes transformation into its standard form, subsequently solved through the solver interface. The standard form of either the primal (default case) or dual problem, can be accessed using the `do_math()` method of the model object. Refer to the sample code below for illustration.

In [9]:
from rsome import ro
import rsome as rso
import numpy as np


n = 3
np.random.seed(1)
c = np.random.normal(size=n)

model = ro.Model()
x = model.dvar(n)

model.max(c @ x)
model.st(rso.norm(x) <= 1)

primal = model.do_math()            # standard form of the primal problem by defualt
dual = model.do_math(primal=False)  # standard form of the dual problem

In [10]:
primal

Conic program object:
Number of variables:           8
Continuous/binaries/integers:  8/0/0
---------------------------------------------
Number of linear constraints:  5
Inequalities/equalities:       2/3
Number of coefficients:        11
---------------------------------------------
Number of SOC constraints:     1
---------------------------------------------
Number of ExpCone constraints: 0
---------------------------------------------
Number of PSCone constraints:  0

In [11]:
dual

Conic program object:
Number of variables:           5
Continuous/binaries/integers:  5/0/0
---------------------------------------------
Number of linear constraints:  4
Inequalities/equalities:       0/4
Number of coefficients:        7
---------------------------------------------
Number of SOC constraints:     1
---------------------------------------------
Number of ExpCone constraints: 0
---------------------------------------------
Number of PSCone constraints:  0

Further insights into the standard format are available through the `show()` method, presenting comprehensive information on variables, objectives, and constraints in a Pandas data frame. 

In [12]:
primal.show()

,x1,x2,x3,x4,x5,x6,x7,x8,sense,constant
Obj,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-
LC1,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,==,-0.0
LC2,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,==,-0.0
LC3,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,==,-0.0
LC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,<=,1.0
LC5,-1.0,-1.624345,0.611756,0.528172,0.0,0.0,0.0,0.0,<=,-0.0
QC1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,-1.0,<=,0.0
UB,inf,inf,inf,inf,inf,inf,inf,inf,-,-
LB,-inf,-inf,-inf,-inf,-inf,-inf,-inf,0.0,-,-
Type,C,C,C,C,C,C,C,C,-,-


In [13]:
dual.show()

,x1,x2,x3,x4,x5,sense,constant
Obj,0.0,0.0,0.0,1.0,0.0,-,-
LC1,0.0,0.0,0.0,0.0,-1.0,==,1.0
LC2,1.0,0.0,0.0,0.0,-1.624345,==,0.0
LC3,0.0,1.0,0.0,0.0,0.611756,==,0.0
LC4,0.0,0.0,1.0,0.0,0.528172,==,0.0
QC1,1.0,1.0,1.0,-1.0,0.0,<=,0.0
UB,inf,inf,inf,inf,0.0,-,-
LB,-inf,-inf,-inf,0.0,-inf,-,-
Type,C,C,C,C,C,-,-


The standard form of a model can be solved by calling the `solve()` method of the `Model` object. The first argument `solver` is used to specify the selected solver interface. If the solver is unspecified, the default linear optimization solver imported from the the `scipy.optimize` module is used for solving the model. RSOME is also compatible with a range of open-source and commercial solvers.  Comprehensive information about the supported solvers is outlined in the table below.

| Solver | RSOME interface | SOCP| ECP | SDP
|:-------|:--------------|:-----------------|:----------------|:------------------------|
|`scipy.optimize`| `lpg_solver` | No | No | No |
|CyLP | `clp_solver` | No | No | No |
|OR-Tools | `ort_solver` | No | No | No |
|ECOS | `eco_solver` | Yes | Yes | No |
|Gurobi | `grb_solver` | Yes | No | No |
|Mosek | `msk_solver` | Yes | Yes | Yes |
|CPLEX| `cpx_solver` | Yes | No | No |
|COPT| `cpt_solver` | Yes | No | No |

The model above involves second-order cone constraints, so we could use ECOS, Gurobi, Mosek, CPLEX, or COPT to solve it. The interfaces for these solvers are imported by the following commands.

In [14]:
from rsome import eco_solver as eco
from rsome import grb_solver as grb
from rsome import msk_solver as msk
from rsome import cpx_solver as cpx
from rsome import cpt_solver as cpt

The interfaces can be then used to attain the solution.

In [15]:
model.solve(eco)

Being solved by ECOS...
Solution status: Optimal solution found
Running time: 0.0004s


In [16]:
model.solve(grb)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-28
Being solved by Gurobi...
Solution status: 2
Running time: 0.0010s


In [17]:
model.solve(grb)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0006s


In [18]:
model.solve(msk)

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0116s


In [19]:
model.solve(cpx)

Being solved by CPLEX...
Solution status: optimal
Running time: 0.0062s


In [20]:
model.solve(cpt)

Cardinal Optimizer v6.5.3. Build date Apr 28 2023
Copyright Cardinal Operations 2023. All Rights Reserved

Being solved by COPT...
Solution status: 1
Running time: 0.0021s


It can be seen that as the model is solved, a three-line message is displayed in terms of 1) the solver used for solving the model; 2) the solution status; and 3) the solution time. This three-line message can be disabled by specifying the second argument `display` to be `False`.

The third argument `params` is used to tune solver parameters. The current RSOME package enables users to adjust parameters for Gurobi, MOSEK, and CPLEX. The `params` argument is a `dict` type object in the format of `{<param1>: <value1>, <param2>: <value2>, <param3>: <value3>, ..., <paramk>: <valuek>}`. Information on solver parameters and their valid values are provided below. Please make sure that you are specifying parameters with the correct data type, otherwise error messages might be raised.
- Gurobi parameters: [https://www.gurobi.com/documentation/current/refman/parameters.html](https://www.gurobi.com/documentation/current/refman/parameters.html)
- MOSEK parameters: [https://docs.mosek.com/latest/pythonapi/parameters.html](https://docs.mosek.com/latest/pythonapi/parameters.html)
- CPLEX parameters: [https://www.ibm.com/docs/en/icos/20.1.0?topic=tutorial-using-cplex-parameters-in-cplex-python-api](https://www.ibm.com/docs/en/icos/20.1.0?topic=tutorial-using-cplex-parameters-in-cplex-python-api)


For example, the following code solves the problem using Gurobi, MOSEK, and CPLEX, respectively, with the relative MIP gap tolerance to be `1e-3`.

In [21]:
model.solve(grb, params={'MIPGap': 1e-3})
model.solve(msk, params={'mioTolRelGap': 1e-3})
model.solve(cpx, params={'mip.tolerances.mipgap': 1e-3})

Being solved by Gurobi...
Solution status: 2
Running time: 0.0010s
Being solved by Mosek...
Solution status: Optimal
Running time: 0.0029s
Being solved by CPLEX...
Solution status: optimal
Running time: 0.0051s


Once the optimization problem is solved, you may use the command `model.get()` to retrieve the optimal objective value. The optimal solution of the variable `x` can be attained as an array by calling `x.get()`. The `get()` method raises an error message if the optimization problem is unsolved, or the optimal solution cannot be found due to infeasibility, unboundedness, or numerical issues. 

(rsome_basics:examples)= 
## Application Examples
- [Mean-Variance Portfolio](example_mean_variance_portfolio.ipynb)
- [Integer Programming for Sudoku](example_sudoku.ipynb)
- [Optimal DC Power Flow](example_dc_opf.ipynb)
- [Conic Relaxation of Optimal AC Power Flows](example_ac_opf.ipynb)
- [The Unit Commitment Problem](example_uc.ipynb)
- [Box with the Maximum Volume](example_max_box.ipynb)
- [Minimal Enclosing Ellipsoid](example_min_ellipsoid.ipynb)
- [Logistic Regression](example_logit.ipynb)

<br>
<br>

<font size="5">Reference</font>

```{bibliography}
:filter: docname in docnames
```